In [ ]:
#!/usr/bin/env python

import sys
sys.path.append('/home/bij/Projects/fdtd/')
import fdtd
import fdtd.backend as bd
import matplotlib.pyplot as plt


# ## Set Backend
#fdtd.set_backend("numpy")
fdtd.set_backend("torch")


# ## Constants
WAVELENGTH = 1550e-9
WAVELENGTH2 = 1550e-8
SPEED_LIGHT: float = 299_792_458.0  # [m/s] speed of light



In [ ]:
for shift in range(0, 14):
    grid = fdtd.Grid(
        (1.5e-5, 1.5e-5, 1),
        grid_spacing=0.1 * WAVELENGTH,
        permittivity=1.0,
        permeability=1.0,
    )
    grid[0:10, :, :] = fdtd.PML(name="pml_xlow")
    grid[-10:, :, :] = fdtd.PML(name="pml_xhigh")
    grid[:, 0:10, :] = fdtd.PML(name="pml_ylow")
    grid[:, -10:, :] = fdtd.PML(name="pml_yhigh")
    grid[:, :, 0] = fdtd.PeriodicBoundary(name="zbounds")
    
    y_mid, x_mid = grid.shape[0]//2, grid.shape[1]//2
    
    grid[y_mid, 25, 0] = fdtd.PointSource(
        period=WAVELENGTH / SPEED_LIGHT, name="linesource0", amplitude=1,
    )
    grid[y_mid, -25, 0] = fdtd.PointSource(
        period=WAVELENGTH / SPEED_LIGHT, name="linesource1", delay=shift,
    )

    grid_E_energies = None
    grid.visualize(z=0, animate=True)
    for i in range(100):
        grid.run(1, progress_bar=False)
        grid_energy_E, grid_energy_H = grid.visualize(z=0, norm='linear', animate=True)
        if(i == 0):
            grid_E_energies = grid_energy_E
        else:
            grid_E_energies += grid_energy_E
        #plt.show()
    plt.imshow(bd.numpy(grid_E_energies))
    plt.savefig('avg_energy_shift_mid={0}.png'.format(shift))


In [ ]:
for shift in range(0, 14):
    shift = 14
    grid = fdtd.Grid(
        (1.5e-5, 1.5e-5, 1),
        grid_spacing=0.1 * WAVELENGTH,
        permittivity=1.0,
        permeability=1.0,
    )
    grid[0:10, :, :] = fdtd.PML(name="pml_xlow")
    grid[-10:, :, :] = fdtd.PML(name="pml_xhigh")
    grid[:, 0:10, :] = fdtd.PML(name="pml_ylow")
    grid[:, -10:, :] = fdtd.PML(name="pml_yhigh")
    grid[:, :, 0] = fdtd.PeriodicBoundary(name="zbounds")
    
    y_mid, x_mid = grid.shape[0]//2, grid.shape[1]//2
    
    grid[y_mid, 25, 0] = fdtd.PointSource(
        period=WAVELENGTH / SPEED_LIGHT, name="linesource0", amplitude=1,
    )
    grid[y_mid, -25, 0] = fdtd.PointSource(
        period=WAVELENGTH / SPEED_LIGHT, name="linesource1", delay=shift,
    )

    
    
    print('Running shift {0}'.format(shift))
    grid.run(1000, progress_bar=False)
    plt.imshow(bd.numpy(grid.E_avg[:, :, 0, :]))
    plt.savefig('avg_energy_shift_mid={0}.png'.format(shift))
    break


In [ ]:
plt.imshow(bd.numpy(bd.sum(grid.E_avg[:, :, 0, :]**2, axis=-1)))

In [ ]:
avg_energy = bd.numpy(bd.sum(grid.E_avg[:, :, 0, :]**2, axis=-1))

In [ ]:
import numpy as np
plt.hist(avg_energy.flatten(), bins=100)

In [ ]:
def NormalizeEnergy(energy, width=3):
    mean = np.mean(energy.flatten())
    std = np.std(energy.flatten())
    h_cutoff = mean + std*width
    energy_normed = (np.clip(energy, 0, h_cutoff))/(h_cutoff)
    return energy_normed

In [ ]:
#plt.hist(avg_energy.flatten(), bins=100)
plt.hist(NormalizeEnergy(avg_energy.flatten()), bins=100)

In [ ]:
NormalizeEnergy(avg_energy.flatten()).min()

In [ ]:
plt.imshow(NormalizeEnergy(avg_energy))